<a href="https://colab.research.google.com/github/edwardoughton/ascend/blob/main/ascend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

Saving mission_minutes_2023.csv to mission_minutes_2023.csv
Saving missions.csv to missions.csv
Saving scenario_costs.csv to scenario_costs.csv


{'mission_minutes_2023.csv': b'use_case,method,mean_minutes_2023\r\nHuman Space Flight,DTE,115860\r\nNear Earth Robotic - LEO Science,DTE,19395\r\nNear Earth Robotic - GEO and Near Earth,DTE,30865\r\nDeep Space Robotic,DTE,0\r\nNear Earth Robotic - Low Latency & Complex Needs,DTE,22562\r\nMission Operations,DTE,0\r\nTerrestrial & Aerial,DTE,0\r\nHuman Space Flight,SR,178044\r\nNear Earth Robotic - LEO Science,SR,151730\r\nNear Earth Robotic - GEO and Near Earth,SR,109\r\nDeep Space Robotic,SR,572\r\nNear Earth Robotic - Low Latency & Complex Needs,SR,4094\r\nMission Operations,SR,0\r\nTerrestrial & Aerial,SR,17344\r\n',
 'missions.csv': b'use_case,type,method,Current,Low,Baseline,High\r\nHuman Space Flight,ADD,DTE,4,2,4,7\r\nNear Earth Robotic - LEO Science,ADD,DTE,34,18,33,58\r\nNear Earth Robotic - GEO and Near Earth,ADD,DTE,11,6,11,19\r\nDeep Space Robotic,ADD,DTE,0,0,0,0\r\nNear Earth Robotic - Low Latency & Complex Needs,ADD,DTE,1,1,1,2\r\nMission Operations,ADD,DTE,0,0,0,0\r\nTer

Load the following data:

* Future missions scenarios
* Mean number of minutes per mission use case
* Cost scenario data

In [ ]:
import pandas as pd

missions_data = pd.read_csv('missions.csv')
missions_data = missions_data.reset_index()
missions_data = pd.melt(
    missions_data,
    id_vars=['use_case','type','method'],
    value_vars=['Current','Low', 'Baseline', 'High'],
    var_name='scenario',
    value_name='missions'
    )
print(missions_data.head(3))
missions_data = missions_data.to_dict('records')

mean_minutes = pd.read_csv('mission_minutes_2023.csv')
print(mean_minutes.head(3))
mean_minutes = mean_minutes.to_dict('records')
mean_minutes
cost_data = pd.read_csv('scenario_costs.csv')
cost_data = cost_data.reset_index()
cost_data = pd.melt(
    cost_data,
    id_vars=['method','other_minutes_sold','cost_scenario'],
    value_vars=['Current','Low', 'Baseline', 'High'],
    var_name='scenario',
    value_name='cost_usd_min'
    )
print(cost_data.head(3))
cost_data = cost_data.to_dict('records')

                                  use_case type method scenario  missions
0                       Human Space Flight  ADD    DTE  Current         4
1         Near Earth Robotic - LEO Science  ADD    DTE  Current        34
2  Near Earth Robotic - GEO and Near Earth  ADD    DTE  Current        11
                                  use_case method  mean_minutes_2023
0                       Human Space Flight    DTE             115860
1         Near Earth Robotic - LEO Science    DTE              19395
2  Near Earth Robotic - GEO and Near Earth    DTE              30865
  method other_minutes_sold cost_scenario scenario  cost_usd_min
0    DTE                 No            S1  Current         102.3
1    DTE                 No            S2  Current         188.4
2    DTE                 No            S3  Current         188.4


Specify a function for returning unit costs, based on various arguments.

In [ ]:
def get_unit_costs(data, method, other_minutes_sold, scenario):
  output = {}
  for item in data:
    if item['method'] == method:
      #10 m for DTE, 50 m for LEO/GEO
      if item['other_minutes_sold'] == other_minutes_sold:
        if item['scenario'] == scenario:
          output[item['cost_scenario']] = item['cost_usd_min']

  return output

print(get_unit_costs(cost_data, 'DTE', 'No', 'Baseline'))

{'S1': 188.4, 'S2': 188.4, 'S3': 188.4, 'S4': 188.4, 'S5': 188.4, 'S6': 188.4}


Estimate results.

In [ ]:
output = []

profit_margin = 1.1 # e.g., 10%

for missions in missions_data:
  for minutes in mean_minutes:
    for other_minutes_sold in ['No','Yes']:
      # print(minutes)
      if missions['use_case'] == minutes['use_case']:
        if missions['method'] == minutes['method']:

          # service split is 50/50 for ADD/FDDN, hence .5 value
          minutes_total = missions['missions'] * minutes['mean_minutes_2023'] * .5

          costs_per_minute = get_unit_costs(
              cost_data,
              missions['method'],
              other_minutes_sold,
              missions['scenario']
              )

          output.append({
              'use_case ': missions['use_case'],
              'type ': missions['type'],
              'method ': missions['method'],
              'other_minutes_sold': other_minutes_sold,
              'scenario ': missions['scenario'],
              'missions': missions['missions'],
              'minutes ': minutes_total,
              'cost_usd_S1': minutes_total * costs_per_minute['S1'] * profit_margin,
              'cost_usd_S2': minutes_total * costs_per_minute['S2'] * profit_margin,
              'cost_usd_S3': minutes_total * costs_per_minute['S3'] * profit_margin,
              'cost_usd_S4': minutes_total * costs_per_minute['S4'] * profit_margin,
              'cost_usd_S5': minutes_total * costs_per_minute['S5'] * profit_margin,
              'cost_usd_S6': minutes_total * costs_per_minute['S6'] * profit_margin,
          })

output = pd.DataFrame(output)
output.to_csv('results.csv', index=False)
